In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, Bounds, OptimizeResult
from CIRModel import BasicCIRModel
import sys; sys.path.insert(0, '..')
from scipy.optimize import Bounds
from scipy.stats import norm

In [5]:
class Calibration:
    """Class for calibrating interest rate models using Maximum Likelihood Estimation (MLE)."""
    
    def __init__(self, data, dt, model_class, initial_params):
        self.data = data
        self.dt = dt
        self.model_class = model_class
        self.initial_params = initial_params

    def _nlog_likelihood(self, params, *args):
        # Create a new dictionary that updates the initial_params with the new parameter values
        updated_params = self.initial_params.copy()
        updated_params.update(dict(zip(args, params)))

        # Initialize the model with the updated parameters
        model = self.model_class(**updated_params)
        loglikelihood = 0
        
        for i in range(len(self.data) - 1):
            current_rate = self.data[i]
            next_rate = self.data[i + 1]
            predicted_rate = model.next_rate(current_rate, self.dt)
            variance = model.sigma**2 * max(current_rate, 0) * self.dt
            probability_density = norm.pdf(next_rate, loc=predicted_rate, scale=np.sqrt(variance))
            loglikelihood -= np.log(probability_density + 1e-10)  # Add a small constant to avoid log(0)

        return loglikelihood

    def calibrate(self, bounds, params):
        # Get initial values for the parameters to be optimized
        initial_values = [self.initial_params[param] for param in params]

        # Minimize the negative log likelihood
        result = minimize(
            self._nlog_likelihood,
            initial_values,
            args=tuple(params),  # Ensure parameters names are passed as a tuple
            method='L-BFGS-B',
            bounds=bounds
        )
        return result

In [7]:
# calibrate using 3mon treasury bill YTM for the past 6 months
df_1_yc = pd.read_csv("DGS3MO.csv")
# data cleaning
df_1_yc.index = pd.to_datetime(df_1_yc.DATE)
df_1_yc = df_1_yc.dropna().drop(columns=["DATE"])
df_1_yc.DGS3MO = df_1_yc.DGS3MO.replace(".", np.nan).astype(float)/100
df_1_yc = df_1_yc.dropna()
# filter out calibration period
df_1_yc = df_1_yc['2023-12-15':'2024-03-15']
data = df_1_yc['DGS3MO'].values

# implementation
calibrator = Calibration(
    data=data,
    dt=1/252,
    model_class=BasicCIRModel,
    initial_params={'kappa': 0.2, 'mu_r': 0.05, 'sigma': 0.01}
)

bounds = Bounds([0.01, 0.01, 0.01], [1, 1, 1])  # Specify parameter bounds
params = ['kappa', 'mu_r', 'sigma']  # Parameters to optimize
result = calibrator.calibrate(bounds=bounds, params=params)

print("Optimized Parameters:")
for param, value in zip(params, result.x):
    print(f"{param}: {value:.4f}")

print(f"\nObjective Function Value: {result.fun:.6f}")
print(f"Success: {result.success}")
print(f"Message: {result.message}")

Optimized Parameters:
kappa: 0.2000
mu_r: 0.0500
sigma: 0.0100

Objective Function Value: -472.265465
Success: True
Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
